In [1]:
!pip install svgpath2mpl
!pip install gym

In [2]:
from collections import namedtuple
import numpy as np
import torch
import torch.nn as nn
import imageio
from svgpath2mpl import parse_path
import matplotlib.pyplot as plt
import matplotlib
from scipy.ndimage import rotate, shift
from matplotlib.animation import FuncAnimation
from environments.wildfire_gym import WildFireGym
from networks.lstm_ppo_net import LSTMPPONet
import torch.nn.functional as F

In [3]:

n_actions = 2
height = width = 100
channels = 2
EPISODES_PER_BATCH = 1
TRAIN_FREQ  = 10
SAVE_FREQ = 10
GAMMA = 0.95
CLIP  = 0.2
N_DRONES = 2

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMPPONet(device,  channels, height, width, n_actions).to(device)
model.load_state_dict(torch.load('ppo_weights.pt'))
model.eval()

LSTMPPONet(
  (fc1): Sequential(
    (0): Linear(in_features=5, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): ReLU()
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): ReLU()
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): ReLU()
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): ReLU()
  )
  (conv): Sequential(
    (0): Conv2d(2, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc2): Sequential(
    (0): Linear(in_features=30976, out_features=500, bias=True)
    (1): ReLU()
    (2): Linear(in_features=500, out_features=100, bias=True)
   

In [5]:
loss = None
i_episode = 1
images = []
steps = 0
wildFireGym = WildFireGym()
observation = wildFireGym.reset()

maps = [None]*N_DRONES
state_vectors = [None]*N_DRONES
hidden = [None]*N_DRONES

while True:
  
  maps[0] = torch.tensor(observation[0]['belief_map'], device=device, dtype=torch.float)
  maps[1] = torch.tensor(observation[1]['belief_map'], device=device, dtype=torch.float)

  state_vectors[0] = torch.tensor(observation[0]['state_vector'], device=device, dtype=torch.float)
  state_vectors[1] = torch.tensor(observation[1]['state_vector'], device=device, dtype=torch.float)
  
  logits1, _, hidden[0] = model(maps[0], state_vectors[0], hidden[0])
  logits2, _, hidden[1] = model(maps[1], state_vectors[1], hidden[1])
  
  policy1 = F.softmax(logits1, dim=1)
  policy2 = F.softmax(logits2, dim=1)
  
  next_observation, rewards, done, _ = wildFireGym.step([ torch.argmax(policy1).item(), torch.argmax(policy2).item()])

  observation = next_observation

  if done:

    break

  observation = next_observation
  steps += 1

  if steps % 5 == 0:

    fig, ax = plt.subplots(1, 5, figsize=(28, 4))
    wildFireGym.render(fig, ax)
    fig.canvas.draw() 
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image  = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    images.append(image)

kwargs_write = {'fps':5.0, 'quantizer':'nq'}
imageio.mimsave('./ppo_example.gif', images, fps=5)

TypeError: reset() takes from 1 to 2 positional arguments but 3 were given